In [7]:
"""Part 1: Alternative Specific Variance

Case Study 5
Demand Modeling: 1.202
Spring 2019
Alexandra Berke
"""

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.draws as draws
import biogeme.models as models

pandas = pd.read_table("swissmetro.dat")
database = db.Database("swissmetro",pandas)

from headers import *

#Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
ASC_CAR_mean = Beta('ASC_CAR_mean',0,-30,30,0)
ASC_CAR_std = Beta('ASC_CAR_std',0,-10,10,1) # fix to zero
ASC_SBB_mean = Beta('ASC_SBB_mean',0,-10,10,1) # fix to zero
ASC_SBB_std = Beta('ASC_SBB_std',0,-10,10,0)
ASC_SM_mean = Beta('ASC_SM_mean',0,-30,30,0)
ASC_SM_std = Beta('ASC_SM_std',0,-10,10,0)
B_CAR_COST	 = Beta('B_CAR_COST',0,-10,10,0)
B_HE	 = Beta('B_HE',0,-10,10,0)
B_SM_COST	 = Beta('B_SM_COST',0,-10,10,0)
B_CAR_TIME	 = Beta('B_CAR_TIME',0,-10,10,0)
B_TRAIN_TIME	 = Beta('B_TRAIN_TIME',0,-10,10,0)
B_SM_TIME	 = Beta('B_SM_TIME',0,-10,10,0)
B_TRAIN_COST	 = Beta('B_TRAIN_COST',0,-10,10,0)
B_SENIOR	 = Beta('B_SENIOR',0,-10,10,0)
B_GA	 = Beta('B_GA',0,-10,80,0)
B_FIRST	 = Beta('B_FIRST',0,-10,80,0)

# Random parameters
# Normalization: First estimate all three, then fix the smallest std to 0 and estimate again
ASC_CAR_random = ASC_CAR_mean + ASC_CAR_std * bioDraws('ASC_CAR_random','NORMAL')
ASC_SBB_random = ASC_SBB_mean + ASC_SBB_std * bioDraws('ASC_SBB_random','NORMAL')
ASC_SM_random = ASC_SM_mean + ASC_SM_std * bioDraws('ASC_SM_random','NORMAL')

# Arithmetic expressions for variables that are not directly available in the data file
SENIOR  = DefineVariable('SENIOR', AGE   ==  5,database )
SM_COST  = DefineVariable('SM_COST', SM_CO   * (  GA   ==  0  ),database)
TRAIN_COST  = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)
one  = DefineVariable('one',1,database)

# Utilities
V_Car_SP = ASC_CAR_random * one + B_CAR_TIME * CAR_TT + B_CAR_COST * CAR_CO + B_SENIOR * SENIOR
V_SBB_SP = ASC_SBB_random * one + B_TRAIN_TIME * TRAIN_TT + B_TRAIN_COST * TRAIN_COST + B_HE * TRAIN_HE \
            + B_GA * GA + B_FIRST * FIRST
V_SM_SP = ASC_SM_random * one + B_SM_TIME * SM_TT + B_SM_COST * SM_COST + B_HE * SM_HE + B_GA * GA \
            + B_SENIOR * SENIOR + B_FIRST * FIRST
V = {3: V_Car_SP,1: V_SBB_SP,2: V_SM_SP}

# Associate the availability conditions with the alternatives
CAR_AV_SP  = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
TRAIN_AV_SP  = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

# Here we use the "biogeme" way for backward compatibility
exclude = (( PURPOSE != 1 ) * (  PURPOSE   !=  3  ) +  ( CHOICE == 0 )) > 0
database.remove(exclude)

# The choice model is a logit, with availability conditions
prob = models.logit(V,av,CHOICE)
logprob = log(MonteCarlo(prob))

# You can change the number of draws
biogeme = bio.BIOGEME(database,logprob,numberOfDraws=2000)

biogeme.modelName = 'Part1_AlternativeSpecificVariance'
results = biogeme.estimate()
print(results)


KeyboardInterrupt: 